# Bus Routes Analysis

This notebook analyzes bus routes in Buenos Aires using GeoJSON data. The objective is to filter routes that go to the city and visualize them.

## Step 1: Load Dependencies and Set Up Map
We begin by importing necessary libraries and creating a base map.

In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from shapely.geometry import MultiLineString
import pandas as pd
from itertools import cycle

## Step 2: Load Bus Route Data
We retrieve data from Buenos Aires' open data portal and inspect the available columns.

In [2]:
bus_data_url = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/colectivos-recorridos/recorrido-colectivo.geojson"
bus_routes = gpd.read_file(bus_data_url)
bus_routes.head(3)

,linea,recorrido,sentido,l_r_s,modalidad,jurisdicci,camara,desde,hasta,geometry
0,130,A,IDA,130AIDA,COMUN,NACIONAL,CETUBA,LA BOCA (CIUDAD AUTONOMA DE BUENOS AIRES),ESTACION BOULOGNE (ex - LINEA BELGRANO NORTE -...,"MULTILINESTRING ((-58.35834 -34.62748, -58.358..."
1,130,A,VUELTA,130AVUELTA,COMUN,NACIONAL,CETUBA,ESTACION BOULOGNE (ex - LINEA BELGRANO NORTE -...,LA BOCA (CIUDAD AUTONOMA DE BUENOS AIRES),"MULTILINESTRING ((-58.56507 -34.50917, -58.565..."
2,092,B,IDA,092BIDA,COMUN,NACIONAL,CETUBA,PLAZA FLORES (CIUDAD AUTONOMA DE BUENOS AIRES),BARRIO TRANSRADIO (PARTIDO DE ESTEBAN ECHEVERR...,"MULTILINESTRING ((-58.46319 -34.62762, -58.462..."


## Step 3: Filtering Routes to the City
We filter the dataset to keep only the routes heading to the city. A manual check is performed on a specific route.

In [3]:
# Define the bus routes of interest and their corresponding colors
target_routes = ["028", "033", "034", "037", "042", "045", "107", "160", "166"]
route_colors = ["green", "limegreen", "cornflowerblue", "mediumspringgreen", "orange", "purple", "blue", "red", "magenta"]
selected_columns = ["linea", "recorrido", "geometry"]
filtered_routes = []

# Filter routes that go to "Ciudad Universitaria"
for route, color in zip(target_routes, route_colors):
    route_data = bus_routes[bus_routes['linea'] == route]

    if route != "166":
        # Apply additional filters based on 'sentido' (direction) and 'hasta' (destination)
        outbound_condition = (route_data['sentido'] == 'IDA') & (route_data['hasta'].str.contains("CIUDAD UNIVERSITARIA"))
        return_condition = (route_data['sentido'] == 'VUELTA') & (route_data['hasta'].str.contains("CIUDAD UNIVERSITARIA"))
    else:
        # Special condition for bus 166
        outbound_condition = (route_data['sentido'] == 'VUELTA') & (route_data['recorrido'] == 'C NUEVO')

    # Store only the selected columns for the filtered routes
    filtered_routes.extend(route_data[(outbound_condition) | (return_condition)][selected_columns].to_dict(orient='records'))


c:\Users\Portables\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopandas\geodataframe.py:1456: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


## Step 4: Visualization of Selected Routes
We plot the chosen bus routes on an interactive map using Folium.

In [6]:
# Create a Folium map centered on Buenos Aires
m = folium.Map(location=[-34.61, -58.38], zoom_start=13)

# Create a dictionary mapping routes to colors
route_color_map = dict(zip(target_routes, route_colors))

# Iterate over the filtered routes and plot them on the map
for route_info in filtered_routes:
    route_geometry = route_info['geometry']
    route_number = route_info['linea']
    route_desc = route_info['recorrido']
    color = route_color_map.get(route_number, 'black')  # Default to black if not found

    # Add the bus route to the map using PolyLine
    folium.PolyLine(
        locations=[(coord[1], coord[0]) for line in route_geometry.geoms for coord in line.coords],
        color=color,
        weight=2.5,
        opacity=1,
        tooltip=f"Route {route_number} {route_desc}",
        popup=f"Route {route_number} {route_desc}"
    ).add_to(m)

# Save the map with legends
m.save('final_map.html')
m